# setup

In [1]:
import os

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

In [2]:
import argparse
import collections
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.swin_transformer_1d as module_arch_swin_transformer
import model.beat_aligned_transformer as module_arch_beat_aligned_transformer

In [3]:
from parse_config import ConfigParser
from trainer import Trainer_beat_aligned_data
from evaluater import Evaluater_beat_aligned_data
from model.metric import ChallengeMetric
from utils.util import load_model
from utils.lr_scheduler import CosineAnnealingWarmUpRestarts, GradualWarmupScheduler
import datetime

In [4]:
files_models = {
    "swin_transformer_1d": ['swin_transformer'],
    "beat_aligned_transformer": ['beat_aligned_transformer'],
}

In [5]:
args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str,
                    help='config file path (default: None)')
args.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default='0', type=str,
                    help='indices of GPUs to enable (default: all)')
args.add_argument('-s', '--seed', type=int, default=0)
args.add_argument('-t', '--only_test', default=False, type=bool,
                    help='only test (default: False)')

_StoreAction(option_strings=['-t', '--only_test'], dest='only_test', nargs=None, const=None, default=False, type=<class 'bool'>, choices=None, help='only test (default: False)', metavar=None)

In [6]:
CustomArgs = collections.namedtuple('CustomArgs', 'flags type target')
options = [
    CustomArgs(['--lr', '--learning_rate'], type=float, target='optimizer;args;lr'),
    CustomArgs(['--bs', '--batch_size'], type=int, target='data_loader;args;batch_size')
]

In [7]:
config = ConfigParser.for_notebook(args, options)

In [8]:
import os
print("torch.cuda.device_count(): ", torch.cuda.device_count())
print("CUDA_VISIBLE_DEVICES: ", os.environ["CUDA_VISIBLE_DEVICES"])

torch.cuda.device_count():  1
CUDA_VISIBLE_DEVICES:  0


# main

In [9]:
logger = config.get_logger('train')

# build model architecture, then print to console
global model
for file, types in files_models.items():
    for type in types:
        if config["arch"]["type"] == type:
            model = config.init_obj('arch', eval("module_arch_" + file))
            logger.info(model)
            if config['arch'].get('weight_path', False):
                model = load_model(model, config["arch"]["weight_path"])

beat_aligned_transformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(8, 95, kernel_size=(1, 5), stride=(1, 5))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      dim=96, input_resolution=80, depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          dim=96, input_resolution=80, num_heads=8, window_size=5, shift_size=0, mlp_ratio=4.0
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            dim=96, window_size=(5, 5), num_heads=8
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(


In [10]:
criterion = getattr(module_loss, config['loss']['type'])
challenge_metrics = ChallengeMetric(config['data_loader']['args']['label_dir'])
metrics = [getattr(challenge_metrics, met) for met in config['metrics']]

Finding label...
Loading labels...
num_files: 43101
Loading weights...
/home/josegfer/li2021bat


In [11]:
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = config.init_obj('optimizer', torch.optim, trainable_params)

In [12]:
if config["lr_scheduler"]["type"] == "CosineAnnealingWarmRestarts":
    params = config["lr_scheduler"]["args"]
    lr_scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=params["T_0"], T_mult=params["T_mult"],
                                                    T_up=params["T_up"], gamma=params["gamma"], eta_max=params["eta_max"])
elif config["lr_scheduler"]["type"] == "GradualWarmupScheduler":
    params = config["lr_scheduler"]["args"]
    scheduler_steplr_args = dict(params["after_scheduler"]["args"])
    scheduler_steplr = getattr(torch.optim.lr_scheduler, params["after_scheduler"]["type"])(optimizer, **scheduler_steplr_args)
    lr_scheduler = GradualWarmupScheduler(optimizer, multiplier=params["multiplier"], total_epoch=params["total_epoch"], after_scheduler=scheduler_steplr)
else:
    lr_scheduler = config.init_obj('lr_scheduler', torch.optim.lr_scheduler, optimizer)

In [13]:
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.valid_data_loader

Loading data...
Finding label...
Loading labels...


100%|██████████| 100/100 [00:00<00:00, 2783.29it/s]

Loading weights...
/home/josegfer/li2021bat
False
False
torch.Size([77, 12, 10, 400]) torch.Size([11, 12, 10, 400])


In [26]:
data_loader.train_dataset.tensors

([tensor([[[[ 7.4000e-02,  7.5960e-02,  8.6467e-02,  ...,  1.4013e-01,
              1.4449e-01,  1.2990e-01],
            [ 9.2000e-02,  9.3181e-02,  1.0531e-01,  ...,  1.7721e-01,
              1.8765e-01,  1.6865e-01],
            [ 4.4000e-02,  4.3339e-02,  5.8099e-02,  ...,  1.8928e-01,
              2.0128e-01,  1.6940e-01],
            ...,
            [ 9.8000e-02,  1.0106e-01,  1.1193e-01,  ...,  1.4730e-01,
              1.5391e-01,  1.4515e-01],
            [ 1.2500e-01,  1.2823e-01,  1.3529e-01,  ...,  1.6158e-01,
              1.5937e-01,  1.5008e-01],
            [ 7.5000e-02,  7.9123e-02,  9.2670e-02,  ...,  1.5832e-01,
              1.6174e-01,  1.4246e-01]],
  
           [[ 9.9000e-02,  1.0168e-01,  1.1693e-01,  ...,  1.9237e-01,
              1.9897e-01,  1.7836e-01],
            [ 1.1400e-01,  1.1612e-01,  1.2833e-01,  ...,  1.9640e-01,
              2.0665e-01,  1.8891e-01],
            [ 5.6000e-02,  5.5393e-02,  7.0729e-02,  ...,  1.9411e-01,
              2.0753

In [31]:
x = data_loader.train_dataset.tensors[0][0]
x2 = data_loader.train_dataset.tensors[0][1]
x.shape, x2.shape

(torch.Size([77, 8, 10, 400]), torch.Size([77, 10]))

In [32]:
y = data_loader.train_dataset.tensors[1]
w = data_loader.train_dataset.tensors[2]
y.shape, w.shape

(torch.Size([77, 108]), torch.Size([77, 108]))